In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

%run AbstractModel.ipynb

In this code, we added 2 more convolutional layers, which can extract more complex features from your accelerometer data. The number of output channels in the convolutional layers gradually increases, as it is common in many deep learning models to gradually increase the complexity and decrease the spatial size.

Please note that the dimension of the input to the fully connected layer depends on the output size of your last convolutional layer. This code assumes that after 3 layers of convolution with kernel size 5 and stride 1, the sequence length is reduced to 82 (from the original 100). You may need to adjust this according to your own situation.

In [ ]:
class JFNet(nn.Module, SKModel):
    def __init__(self, sequence_length):
        super(JFNet, self).__init__()
        # assertion
        SKDescriptors.validate_class_type(classtype)
        # needed definitions
        self.classtype = classtype
        self.sequence_length = sequence_length
        input_num = SKDescriptors.NUM_OF_INPUTS_PER_TYPE[self.classtype]

        self.conv1 = nn.Conv1d(input_num, 64, kernel_size=3)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3)
        
        # Adjust the fully connected layer's input size based on the new sequence length after convolutions.
        # Adjusted for sequence length = 4 after 3 conv layers with kernel size 3
        # 10 -3 + 1 = 8 after the first layer
        # 8 - 3 + 1 = 6 after the second layer
        # 6 - 3 + 1 = 4 after the third layer
        self.fc1 = nn.Linear(256 * (self.sequence_length - 6), 128)  # Adjusted for sequence length = 4 after 3 conv layers with kernel size 3
        self.fc2 = nn.Linear(128, SKDescriptors.NUM_OF_CLASSES_PER_TYPE[self.classtype])

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(-1, self.num_flat_features(x))  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  # Apply softmax to the output layer

    def num_flat_features(self, x):
        size = x.size()[1:]  # All dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    def skm_train(self):
        # Instantiate the network and optimizer
        # net = Net()
        optimizer = optim.SGD(self.parameters(), lr=0.01)

        # Define the loss function
        criterion = nn.CrossEntropyLoss()

        # Assume we have a data loader `train_dataloader` which loads our training accelerometer data
        for epoch in range(1001):  # loop over the dataset multiple times

            running_loss = 0.0
            for i, data in enumerate(train_dataloader, 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                # print statistics
                running_loss += loss.item()
                if (epoch % 200) == 0:    # print every 2000 mini-batches
                    print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / 2000))
                    running_loss = 0.0

        print('Finished Training')

        # Now we will validate the model using test data
        correct = 0
        total = 0

        with torch.no_grad():   # Since we're not training, we don't need to calculate the gradients
            for data in test_dataloader:
                inputs, labels = data
                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the test data: %d %%' % (100 * correct / total))



        # Saving the entire model

        torch.save(net, 'model.pth')
